In [6]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict

if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [7]:
env = BlackjackEnv()

In [8]:
def mc_prediction(policy, env, num_episodes, discount_factor=1.0):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given policy using sampling.
    
    Args:
        policy: A function that maps an observation to action probabilities.
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # The final value function
    V = defaultdict(float)
    
    # Implement this!
    for i_episodes in range(num_episodes):
        if i_episodes % 1000 == 0:
            print("\rEpisodes {}/{}".format(i_episodes, num_episodes))
            sys.stdout.flush()
        
        # Generate aan episodes
        episodes = []
        observation = env.reset()
        for t in range(100):
            action = policy(observation)
            next_state, reward, done, _ = env.step(action)
            episodes.append((state, action, reward))
            if done:
                break
            observation = next_state
            
        state_in_episodes = set([tuple(x[0]) for x in episodes])
        for state in state_in_episodes:
            first_occurency_idx = next(i for i, x in enumerate(episodes) if x[0] == state)
            # sum up all rewards since the first occurance
            G = sum(x[2]*discount_factor**i for i, x in enumerate(episodes[first_occurency_idx:]))
            # calculate average return for this state over all smapled episodes
            returns_sum[state] += G
            returns_count[state] += 1.0
            V[state] = returns_sum[state] / returns_count[state] 
    return V    

In [9]:
def sample_policy(observation):
    """
    A policy that sticks if the player score is > 20 and hits otherwise.
    """
    score, dealer_score, usable_ace = observation
    return np.array([1.0, 0.0]) if score >= 20 else np.array([0.0, 1.0])

In [10]:
V_10k = mc_prediction(sample_policy, env, num_episodes=10000)
plotting.plot_value_function(V_10k, title="10,000 Steps")

V_500k = mc_prediction(sample_policy, env, num_episodes=500000)
plotting.plot_value_function(V_500k, title="500,000 Steps")

Episodes 0/10000


AssertionError: 